# Ex. 04 - Simulazione di Dinamica Molecolare - ensemble NVE

## Introduzione

In questo esercizio è stato utilizzato un codice di dinamica molecolare (MD) per simulare l'evoluzione temporale di un piccolo sistema "classico" di $108$ particelle - confinate in una scatola - interagenti tra loro con un potenziale a due corpi a corto range del tipo Lennard-Jones $V_{L-J}(r)$:

$$V_{L-J}(r)=4\epsilon[(\sigma/r)^{12}-(\sigma/r)^6]$$

Il sistema è stato studiato all'equilibrio termodinamico e con i vincoli che definiscono l'ensemble microcanonico: energia totale ($E$), volume ($V$) e numero di particelle ($N$) costanti nel tempo.

Sono state effettuate simulazioni per tre diverse fasi termodinamiche del sistema (solida, liquida e gassosa) e sono stati calcolati i valori istantanei e mediati nel tempo di alcune grandezze termodinamiche di interesse in funzione del tempo di integrazione. 

### Le unità di  misura
Nonostante i risultati siano stati presentati nelle unità del SI, non è detto che queste siano le più adatte a descrivere l'evoluzione della dinamica - a livello microscopico - di un sistema. È opportuno invece individuare le scale di (ad esempio) energia, lunghezza e massa tipiche del sistema in esame ed esprimere le altre grandezze in funzione di quest'ultime. L'adozione delle unità $\textit naturali$ o $\textit ridotte$ comporta due vantaggi:

- i valori delle grandezze espressi in termini di queste unità saranno dell'ordine dell'unità (o di poco inferiori/superiori). Questo permette di evitare - a livello computazionale  - i problemi di under/overflow e di monitorare facilmente i valori calcolati delle diverse grandezze
- uno stesso stato in unità ridotte può rappresentare molteplici sistemi fisici, caratterizzati cioè da diverse scale di energia, massa e lunghezza (legge di corrispondenza degli stati)

In questo caso, le unità naturali sono quelle che compaiono nel potenziale e nelle eq. del moto:
- $\epsilon$ per le energie
- $\sigma$ per le lunghezze
- la massa $m$ di una particella per le masse

Le altre grandezze possono espresse come combinazione di queste tre:
- tempo $\tau$: $\sigma m^{1/2}\epsilon{-1/2}$
- velocità v: $\epsilon^{1/2}m^{-1/2}$
- pressione p: $\epsilon\sigma^{-3}$
- densità numero $\rho$: $\sigma^{-3}$

Le simulazioni sono state quindi condotte utilizzando le unità ridotte e solo in seguito le varie grandezze sono state riscalate a seconda del sistema fisico in esame. In particolare, sono stati presentati i risultati per l'Argon e il Krypton, entrambi ben descritti da un potenziale di tipo L-J e caratterizzati dai seguenti valori di $\sigma,\epsilon,m$:
- Argon $\sigma = 0.34$ nm, $\epsilon/k_B = 120$ K, $m=39.948$ amu
- Krypton: $\sigma = 0.364$ nm, $\epsilon/k_B = 164$ K, $m=83.798$ amu

### L'algoritmo di integrazione
Alla base dell'evoluzione temporale di un esperimento di MD vi è l'integrazione delle equazioni del moto, che dev'essere effettuata mediante l'uso di un apposito metodo di integrazione alle differenze finite. In questo caso è stato utilizzato l'algoritmo Verlet, che permette di calcolare la posizione di una particella $r(t+\delta t)$ al tempo $t+\delta t$ usando le posizioni della particella ai tempi $t$ e $t-\delta t$. Questo algoritmo è stato scelto per la sua efficienza e accuratezza (l'errore su $r(t+\delta t) \sim O(\delta t^3)$) e perchè conserva bene l'energia totale anche per un time step abbastanza grande. La conservazione dell'energia è infatti una condizione necessaria e deve essere verificata lungo tutto il corso della simulazione. Per soddisfarla, è necessario scegliere un corretto time step di integrazione, che può dipendere dall'andamento del potenziale, dalla temperatura (cioè dalla velocità media delle particelle), dalla massa delle particelle ecc. A seconda della fase termodinamica simulata, può essere necessario adottare un time step differente, ma è stato verificato che per simulazioni di $10^4$ passi, con un singolo passo di $0.0005$ (periodo di integrazione totale $\sim 1$ in unità ridotte), la conservazione dell'energia è garantita e la dinamica è "fluida", cioè priva di cambiamenti bruschi.

NB: l'algoritmo Verlet non richiede la conoscenza delle velocità al tempo $t$ per calcolare le posizioni al tempo $t+dt$, ma richiede invece le posizioni al tempo $t$ e $t-dt$. Per poter iniziare la simulazione sono stati implementati due metodi:

- uso dell'algoritmo Eulero per generare la posizione al tempo $-dt$ a partire da quella al tempo $0$
- salvataggio, al termine di una simulazione, della posizione finale (in config.final) e di quella precedente (in config.old) per poter ricominciare una nuova simulazione


Poichè le particelle, a seconda dello stato termodinamico, sono state confinate in una scatola a volume fissato, è stato necessario implementare le condizioni periodiche (PBC) al contorno per poter eliminare gli effetti di bordo (non trascurabili in sistemi così piccoli). L'adozione delle PBC, inoltre, ha richiesto l'uso di un raggio di cut-off $r_c$, necesario anche per ridurre il tempo di calcolo. In questo modo, è stata calcolata l'energia di interazione tra atomi aventi distanza reciproca minore di $r_c$. Questa approssimazione è giustificata dalla presenza di un potenziale a corto range e la sua validità può essere verificata (ma qui non sarà fatto) calcolando il contributo medio del potenziale nel range trascurato [$r_c, \infty)$.

### Le grandezze misurate
Sono stati calcolati i valori istantanei e mediati nel tempo delle seguenti grandezze, qui espresse in unità ridotte:

- Energia potenziale: $U = \sum_{i<j}^N V_{L-J}(|\vec r_i-\vec r_j|)$
- Energia cinetica: $K=\frac{1}{2}\sum_i^N \dot r_i^2 $
- Energia totale: $E=U+K$
- Temperatura ("istantanea"): $T=\frac{2}{3}\frac{K}{N}$
- Pressione ("istantanea"): $P = \rho T + \frac{1}{3V} \sum_{i<j1}^{N}48 \left[ \left(\frac{1}{|\vec{r}_i -\vec{r}_j|}\right)^{12} - 
\frac{1}{2} \left(\frac{1}{|\vec{r}_i -\vec{r}_j|}\right)^6 \right]  $

Quello che ci si aspetta in un sistema NVE all'equilibrio termodinamico è che l'energia sia conservata nel tempo (questo anche fuori dall'equilibrio), mentre le altre quantità fluttuino, nel limite termodinamico, attorno ad un valor medio. Per verificare che l'energia totale sia conservata è necessario che le fluttuazioni di quest'ultima siano trascurabili se confrontate con quelle delle altre grandezze. 

Le medie temporali, che all'equilibrio coincidono con le medie di ensemble, sono state effettuate con il metodo del blocking, che ha anche permesso di calcolare gli errori. La taglia di ciascun blocco è stata scelta, per via empirica, in maniera tale da essere maggiore del tempo di correlazione delle varie grandezze ($100$ passi temporali). 

I grafici delle varie grandezze in funzione del tempo di integrazione (numero di blocchi nel caso delle medie) per le diverse fasi termodinamiche sono stati riportati nei jupyter "Solid.ipynb","Liquid.ipynb","Gas.ipynb".

Le quantità termodinamiche misurate - all'equilibrio - durante la simulazione sono medie di funzioni degli stati microscopici del sistema. 
Esse sono quindi grandezze macroscopiche, che non danno informazioni dirette sulla struttura microscopica del sistema. Una grandezza che invece riflette in maniera diretta lo stato microscopico del sistema (per quanto riguarda le posizioni degli atomi) e che permette di distinguere le tre fasi termodinamiche da un punto di vista strutturale è la funzione di distribuzione radiale (o a due corpi) $g(\vec r_i,\vec r_j)$. In un fluido omogeneo $g(\vec r_1,\vec r_2)=g(|\vec r_1-\vec r_2|)=g(r)$, cioè dipende solo dalla distanza $r$ tra due corpi, e rappresenta la probabilità di trovare una coppia di atomi posti ad una distanza $r$ tra loro (normalizzata rispetto alla stessa probabilità in un sistema alla stessa densità ma con distribuzione casuale degli atomi). Si definisce nel seguente modo (per un sistema di $N$ particelle):

$$g_N^{(2)}(\vec r_1,\vec r_2)=\frac{\rho_N^{(2)}(\vec r_1,\vec r_2)}{\prod_{i=1}^2\rho_N^{(1)}(\vec r_i)}$$

dove:

$$\rho_N^{(n)}(\vec r_1,...,\vec r_n)=\frac{N!}{(N-n)!}\int d\vec r_{n+1}...d\vec r_N p(\vec r_1,...,\vec r_N)$$

e $p(\vec r_1,...,\vec r_N))$ è la distribuzione di probabilità delle posizioni degli atomi.

Per poter calcolare la $g(r)$ durante il corso della simulazione si utilizza la seguente espressione, ricavata a lezione:

$$g(r)=\frac{1}{\rho N \Delta V(r)}\big \langle \sum_{i<j}^N \delta(r-|\vec r_i-\vec r_j|)\big \rangle$$

dove $\rho$ è la densità numero e $\Delta V(r)=\frac{4\pi}{3}[(r+dr)^3-r^3]$. Operativamente, quindi, la $g(r)$ si costruisce riempiendo un istogramma con bin di dimensione $dr$: il bin a $r$ viene incrementato di $2$ ogni volta che una coppia di atomi si trova a distanza compresa tra $r$ e $r+dr$. L'istogramma viene in seguito normalizzato con il fattore $\rho N \Delta V(r)$.

La funzione di distribuzione radiale permette di calcolare il valor medio di una qualunque funzione a due corpi delle posizioni e, quindi, ad esempio, permette di stimare il contributo del potenziale trascurato dal cut-off. 

I grafici delle funzioni di distribuzione radiale per le tre fasi termodinamiche e i relativi commenti sono riportati nel jupyer "GdR.ipynb". 

### Le 3 fasi termodinamiche e l'equilibrazione 
Il sistema di $108$ particelle con interazione Lennard Jones è stato studiato per 3 diverse fasi termodinamiche - solida, liquida e gassosa - caratterizzate dai parametri temperatura e densità media. Anche il raggio di cut-off è stato scelto in base alla fase. Nella simualazione sono stati usati i seguenti parametri (espressi in unità ridotte):

- fase solida: $\rho^\star = 1.1$, $T^\star = 0.8$, $r_c = 2.2$
- fase liquida: $\rho^\star = 0.8$, $T^\star = 1.1$, $r_c = 2.5$
- fase gassosa: $\rho^\star = 0.05$, $T^\star = 1.2$, $r_c = 5.0$

Per controllare la densità media è stato sufficiente regolare la taglia della scatola, dato che il numero di particelle è rimasto invariato per tutte e tre le fasi. 
Impostare la temperatura desiderata (e quindi ottenere l'equilibrio termodinamico) è stato, invece, più complesso. Il sistema studiato, infatti, conserva energia totale, volume e numero di particelle, ma non la temperatura. Quest'ultima, inoltre, non è nemmeno definita nelle prime fasi della simulazione, quando il sistema è fuori dall'equilibrio.
Per impostare una "temperatura" iniziale sono state settate le velocità iniziali in modo che l'energia cinetica media per particella $k$ fosse pari a $\frac{3}{2}T$. Si è quindi osservato che, a seconda delle condizioni iniziali, durante la prima fase della simulazione la temperatura variava bruscamente e solo dopo alcuni step di integrazione raggiungeva un valore approssimativamente costante (quello di equilibrio), ma diverso da quello impostato inizialmente. 

Per equilibrare correttamente il sistema si può impostare - per tentativi - una temperatura iniziale che poi permetta, attraverso l'evoluzione temporale, di giungere alla temperatura desiderata. Altrimenti, ed è quello che è stato fatto qui - si può implementare un algoritmo che permette di riscalare periodicamente la velocità delle particelle in maniera che l'energia cinetica media per particella corrisponda alla temperatura desiderata. Ad ogni riscalamento delle velocità al tempo $t$ segue il calcolo delle "nuove" posizioni al tempo $t-dt$ consistenti con le nuove velocità: $r(t-dt)=r(t+dt)-v_{new}(t)dt$. Nei jupyter "Solid.ipynb","Liquid.ipynb" e "Gas.ipynb" sono mostrati i grafici della temperatura in funzione del tempo durante la fase di equilibrazione.